In [1]:
!pip install openai

     ---------------------------------------- 0.0/90.6 kB ? eta -:--:--
     --------------------------- ------------ 61.4/90.6 kB 1.6 MB/s eta 0:00:01
     ---------------------------------------- 90.6/90.6 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   --- ------------------------------------ 0.1/1.1 MB 5.1 MB/s eta 0:00:01
   ------ --------------------------------- 0.2/1.1 MB 2.6 MB/s eta 0:00:01
   ----------- ---------------------------- 0.3/1.1 MB 2.5 MB/s eta 0:00:01
   ------------- -------------------------- 0.3/1.1 MB 2.4 MB/s eta 0:00:01
   -------------------- ------------------- 0.5/1.1 MB 2.6 MB/s eta 0:00:01
   -------------------------- ------------- 0.7/1.1 MB 2.8 MB/s eta 0:00:01
   ---------------------------------- ----- 0.9/1.1 MB 2.9 MB/s eta 0:00:01
   ---------------------------------------  1.1/1.1 MB 2.9 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 2.9 MB/s eta 0:00:00
   ------------


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\USERAS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv
from typing import List, Dict, Any
import httpx
from datetime import datetime
import re

load_dotenv()

True

In [2]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

In [70]:
response = client.chat.completions.create(
  model="openai/gpt-oss-20b:free",
  messages=[
          {
            "role": "user",
            "content": "Who won mayor election in new york city in 2025"
          }
        ],
  extra_body={"reasoning": {"enabled": True}}
)

response = response.choices[0].message
print(response.content)

I’m sorry, but I don’t have information on the 2025 New York City mayoral election. My training only goes up to 2021, and I can’t provide details about events that occurred after that. For the most accurate and up‑to‑date results, I recommend checking reputable news outlets, the official New York City Board of Elections website, or other reliable sources.


In [3]:
# To support actions, we need a standard way to represent tools
class Tool:
    def __init__(self, name: str, func: callable, description: str):
        self.name = name
        self.func = func
        self.description = description #The agent uses this to decide whether to call it

    def run(self, inputQ):
        return self.func(inputQ)

In [49]:
class ReActAgent:
    def __init__(self, client: OpenAI, system: str, tools: List[Tool]):
        self.client = client
        # System contains the specific instructions for the agent
        self.system = system
        self.tools = tools
        # To store the sent messages and responses (so that we can use it furthur), initially we are storing the system message
        self.messages: List[Dict[str, str]] = [{"role": "developer", "content": system}]
        self.max_iterations = 5

    def _get_completion(self):
        completion = self.client.responses.create(
            model="openai/gpt-oss-120b:free",
            input=self.messages,
            reasoning={"effort": "medium"},
            extra_body={"reasoning": {"enabled": True}}
        )
        return completion.output_text


    def run(self, query: str):
        # Adds user query to the conversation history
        self.messages.append({"role": "user", "content": query})

        for _ in range(self.max_iterations):
          # Each response can be a thought, an action or an answer
            response = self._get_completion()
            print(f"Agent: {response}")
          # Let's try to find whether the response contains any action
          # () denotes a new group, w means any group, s* arbitary whitesapce. Dotall to see multi line outputs
            action_match = re.search(
                r"""
                Action        # the word Action
                \s*[:：]\s*   # colon (normal or unicode), flexible spacing
                [*_\-]*\s*   # optional markdown junk like ** ** __ --
                ([^\[\n*]+?) # group 1: tool name
                \s*\[\s*     # opening bracket
                ([^\]]+?)    # group 2: tool input
                \s*\]        # closing bracket
                """,
                response,
                re.IGNORECASE | re.VERBOSE
            )

            if action_match:
                tool_name, tool_input = map(str.strip, action_match.groups())
                # print("Action_matched")
                print("*************" + tool_name, tool_input)
                observation = self._execute_tool(tool_name, tool_input)
                print(f"Observation: {observation}")
                self.messages.append({
                      "role": "assistant",
                      "content": f"Observation: {observation}"
                  })
            elif "Answer:" in response:
                return response.split("Answer:", 1)[1].strip()

        return "Max iterations reached without finding an answer."



    def _execute_tool(self, tool_name: str, tool_input: str):
      print("" + tool_name + ": " + tool_input)
    #   if not (tool_input.startswith('"') and tool_input.endswith('"')):
    #     if not (tool_name.lower() == "wikipedia"):
    #         tool_input = f'"{tool_input}"'

      for tool in self.tools:
          if tool.name.lower() == tool_name.lower():
              return tool.run(tool_input)

      return f"Tool '{tool_name}' not found."



In [50]:
def wikipedia_search(query: str):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    },
    headers={
            "User-Agent": "MyPythonApp/1.0 (https://example.com/)"
        })
    return response.json()["query"]["search"][0]["snippet"]

def calculate(expression: str):
  print(expression)
  return str(eval(expression)) #evaluates python expression

def current_date():
    return datetime.now().strftime("%Y-%m-%d")

tools = [
    Tool("Wikipedia", wikipedia_search, "Search Wikipedia for information"),
    Tool("Calculate", calculate, "Perform mathematical calculations"),
    Tool("CurrentDate", current_date, "Get the current date")
]

In [51]:
calculate("2*2")

2*2


'4'

In [64]:
system_prompt = """
You are an AI assistant using the ReAct (Reasoning and Acting)  to answer questions.
Your thought process should follow this loop: Thought, Action, Observation.

Available tools:
1. Wikipedia[query]: Search Wikipedia for information
2. Calculate[expression]: Perform mathematical calculations
3. CurrentDate[]: Get the current date


For each step:
1. Thought: Explain your reasoning and next steps.
2. Action: Use one of the available tools in the format Tool[input].
3. Observation: Wait for the result of the action and look for probable answer in the result. For Wikipedia ignore the tags and find useful information from texts.

Repeat this process until you have enough information to provide a final answer.
When you're ready to give the final answer, start your response with "Answer:".

Example 1:
Human: Name the top 5 highest-grossing movies.

AI: Thought: To answer this question, I need to find out which movies have the highest net gross earnings. Let's start by searching for a list of the top 5 highest-grossing movies.

Action: Wikipedia[highest-grossing movies]

Example 2:
Human: What is the value of 2*2?

AI: The user asks for the result of 2 multiplied by 2. I can compute this directly using the Calculate tool.

Action: Calculate[2*2]

Observation: 4

Answer: 4
"""

In [69]:
agent = ReActAgent(client, system_prompt, tools)
query = "Calculate 5/7+2"
result = agent.run(query)
print(result)

Agent: Action: Calculate[5/7 + 2]
*************Calculate 5/7 + 2
Calculate: 5/7 + 2
5/7 + 2
Observation: 2.7142857142857144
Agent: Answer: 2.7142857142857144
2.7142857142857144


In [11]:
agent.messages

[{'role': 'system',
  'content': '\nYou are an AI assistant using the ReAct (Reasoning and Acting)  to answer questions.\nYour thought process should follow this loop: Thought, Action, Observation.\n\nAvailable tools:\n1. Wikipedia[query]: Search Wikipedia for information\n2. Calculate[expression]: Perform mathematical calculations\n3. CurrentDate[]: Get the current date\n\n\nFor each step:\n1. Thought: Explain your reasoning and next steps.\n2. Action: Use one of the available tools in the format Tool[input].\n3. Observation: Wait for the result of the action.\n\nRepeat this process until you have enough information to provide a final answer.\nWhen you\'re ready to give the final answer, start your response with "Answer:".\n\nExample:\nHuman: Name the top 5 highest-grossing movies.\n\nAI: Thought: To answer this question, I need to find out which movies have the highest net gross earnings. Let\'s start by searching for a list of the top 5 highest-grossing movies.\n\nAction: Wikipedia[

In [17]:
messages: List[Dict[str, str]] = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": query})

completion = client.responses.create(
    model="openai/gpt-oss-120b:free",
    input=messages,
    reasoning={"effort": "medium"},
    extra_body={"reasoning": {"enabled": True}}
)

print(completion)


Response(id='gen-1765649703-bEfXKw1l21TpIAHYWbXK', created_at=1765649706.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='openai/gpt-oss-120b:free', object='response', output=[ResponseReasoningItem(id='rs_tmp_wtg1x9nfwme', summary=[], type='reasoning', content=[Content(text='The user asks: "Who won the tiitle for 2025 Formula One World Championship?" Likely they mean "title". The 2025 F1 season hasn\'t happened yet (as of current date 2025-12-13). Actually current date is 2025-12-13, so the 2025 season would have concluded by now. Need to find who won the 2025 F1 World Drivers\' Championship. Use Wikipedia.We need to search Wikipedia for "2025 Formula One World Championship".', type='reasoning_text')], encrypted_content=None, status=None), ResponseOutputMessage(id='msg_tmp_i0x6vvub5yo', content=[ResponseOutputText(annotations=[], text="**Thought:** I need to find out who won the 2025 Formula One World Drivers' Championship. I'll search Wikipedia for the 20

Response(id='gen-1765649703-bEfXKw1l21TpIAHYWbXK', created_at=1765649706.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='openai/gpt-oss-120b:free', object='response', output=[ResponseReasoningItem(id='rs_tmp_wtg1x9nfwme', summary=[], type='reasoning', content=[Content(text='The user asks: "Who won the tiitle for 2025 Formula One World Championship?" Likely they mean "title". The 2025 F1 season hasn\'t happened yet (as of current date 2025-12-13). Actually current date is 2025-12-13, so the 2025 season would have concluded by now. Need to find who won the 2025 F1 World Drivers\' Championship. Use Wikipedia.We need to search Wikipedia for "2025 Formula One World Championship".', type='reasoning_text')], encrypted_content=None, status=None), ResponseOutputMessage(id='msg_tmp_i0x6vvub5yo', content=[ResponseOutputText(annotations=[], text="**Thought:** I need to find out who won the 2025 Formula One World Drivers' Championship. I'll search Wikipedia for the 2025 Formula One season results.\n\n**Action:** Wikipedia[2025 Formula One World Championship]", type='output_text', logprobs=None)], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=None, tool_choice='auto', tools=[], top_p=None, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, prompt_cache_retention=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None, enabled=True), safety_identifier=None, service_tier='auto', status=None, text=None, top_logprobs=None, truncation=None, usage=ResponseUsage(input_tokens=319, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=168, output_tokens_details=OutputTokensDetails(reasoning_tokens=102), total_tokens=487, cost=0, is_byok=False, cost_details={'upstream_inference_cost': None, 'upstream_inference_input_cost': 0, 'upstream_inference_output_cost': 0}), user=None, output_text='', store=False)

In [31]:
def wikipedia_search(query: str):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    },
    headers={
            "User-Agent": "MyPythonApp/1.0 (https://example.com/)"
        })
    return response.json()["query"]["search"]

In [26]:
!pip install bs4

   ---------------------------------------- 0.0/107.7 kB ? eta -:--:--
   ---------------------------------------- 107.7/107.7 kB 3.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\USERAS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [27]:
from bs4 import BeautifulSoup

In [36]:
input = "2025 Formula One World Championship"
# input = f'"{input}"'
print(input)

# soup = BeautifulSoup(wikipedia_search(input), "html.parser")
# pretty_html = soup.prettify()
print(wikipedia_search(input))

2025 Formula One World Championship
[{'ns': 0, 'title': '2025 Formula One World Championship', 'pageid': 72768531, 'size': 184920, 'wordcount': 10306, 'snippet': 'Support series: <span class="searchmatch">Formula</span> 2 <span class="searchmatch">Championship</span> FIA <span class="searchmatch">Formula</span> 3 <span class="searchmatch">Championship</span> F1 Academy Porsche Supercup The <span class="searchmatch">2025</span> FIA <span class="searchmatch">Formula</span> <span class="searchmatch">One</span> <span class="searchmatch">World</span> <span class="searchmatch">Championship</span> was a motor racing', 'timestamp': '2025-12-12T20:10:00Z'}, {'ns': 0, 'title': "List of Formula One World Drivers' Champions", 'pageid': 3437295, 'size': 87490, 'wordcount': 2422, 'snippet': 'cases on closed city streets. The <span class="searchmatch">World</span> Drivers&#039; <span class="searchmatch">Championship</span> is presented by the FIA to the most successful <span class="searchmatch">Formu

[{'ns': 0, 'title': '2025 Formula One World Championship', 'pageid': 72768531, 'size': 184920, 'wordcount': 10306, 'snippet': 'Support series: <span class="searchmatch">Formula</span> 2 <span class="searchmatch">Championship</span> FIA <span class="searchmatch">Formula</span> 3 <span class="searchmatch">Championship</span> F1 Academy Porsche Supercup The <span class="searchmatch">2025</span> FIA <span class="searchmatch">Formula</span> <span class="searchmatch">One</span> <span class="searchmatch">World</span> <span class="searchmatch">Championship</span> was a motor racing', 'timestamp': '2025-12-12T20:10:00Z'}, {'ns': 0, 'title': "List of Formula One World Drivers' Champions", 'pageid': 3437295, 'size': 87490, 'wordcount': 2422, 'snippet': 'cases on closed city streets. The <span class="searchmatch">World</span> Drivers&#039; <span class="searchmatch">Championship</span> is presented by the FIA to the most successful <span class="searchmatch">Formula</span> <span class="searchmatch">One</span> driver over the course of the season', 'timestamp': '2025-12-13T15:43:46Z'}, {'ns': 0, 'title': 'Formula One', 'pageid': 10854, 'size': 214652, 'wordcount': 18723, 'snippet': 'Internationale de l&#039;Automobile (FIA). The FIA <span class="searchmatch">Formula</span> <span class="searchmatch">One</span> <span class="searchmatch">World</span> <span class="searchmatch">Championship</span> has been <span class="searchmatch">one</span> of the <span class="searchmatch">world&#039;s</span> premier forms of motorsport since its inaugural', 'timestamp': '2025-12-13T15:18:22Z'}, {'ns': 0, 'title': '2024 Formula One World Championship', 'pageid': 72504121, 'size': 222125, 'wordcount': 13568, 'snippet': '2024 FIA <span class="searchmatch">Formula</span> <span class="searchmatch">One</span> <span class="searchmatch">World</span> <span class="searchmatch">Championship</span> Drivers&#039; Champion: Max Verstappen Constructors&#039; Champion: McLaren-Mercedes Previous 2023 Next <span class="searchmatch">2025</span> Races by country', 'timestamp': '2025-12-12T10:33:31Z'}, {'ns': 0, 'title': '2025 Formula 2 Championship', 'pageid': 75724260, 'size': 135944, 'wordcount': 7457, 'snippet': 'series: <span class="searchmatch">Formula</span> <span class="searchmatch">One</span> <span class="searchmatch">World</span> <span class="searchmatch">Championship</span> Support series: FIA <span class="searchmatch">Formula</span> 3 <span class="searchmatch">Championship</span> F1 Academy Porsche Supercup The <span class="searchmatch">2025</span> FIA <span class="searchmatch">Formula</span> 2 <span class="searchmatch">Championship</span> was a', 'timestamp': '2025-12-07T19:09:04Z'}, {'ns': 0, 'title': '2025–26 Formula E World Championship', 'pageid': 80182163, 'size': 43065, 'wordcount': 2212, 'snippet': '<span class="searchmatch">2025</span>–26 FIA <span class="searchmatch">Formula</span> E <span class="searchmatch">World</span> <span class="searchmatch">Championship</span> Previous 2024–25 Next 2026-27 The <span class="searchmatch">2025</span>–26 ABB FIA <span class="searchmatch">Formula</span> E <span class="searchmatch">World</span> <span class="searchmatch">Championship</span> is the twelfth season of the FIA', 'timestamp': '2025-12-11T13:10:58Z'}, {'ns': 0, 'title': '2026 Formula One World Championship', 'pageid': 71681684, 'size': 59598, 'wordcount': 4018, 'snippet': '2026 FIA <span class="searchmatch">Formula</span> <span class="searchmatch">One</span> <span class="searchmatch">World</span> <span class="searchmatch">Championship</span> Previous <span class="searchmatch">2025</span> Next 2027 Races by country Races by venue Support series: <span class="searchmatch">Formula</span> 2 <span class="searchmatch">Championship</span> FIA <span class="searchmatch">Formula</span> 3 Championship', 'timestamp': '2025-12-12T07:55:19Z'}, {'ns': 0, 'title': '2023 Formula One World Championship', 'pageid': 69634872, 'size': 226383, 'wordcount': 14105, 'snippet': 'series: <span class="searchmatch">Formula</span> 2 <span class="searchmatch">Championship</span> FIA <span class="searchmatch">Formula</span> 3 <span class="searchmatch">Championship</span> Porsche Supercup The 2023 FIA <span class="searchmatch">Formula</span> <span class="searchmatch">One</span> <span class="searchmatch">World</span> <span class="searchmatch">Championship</span> was a motor racing <span class="searchmatch">championship</span> for', 'timestamp': '2025-12-10T04:55:26Z'}, {'ns': 0, 'title': '2025 FIA Formula 3 Championship', 'pageid': 75724265, 'size': 128678, 'wordcount': 6916, 'snippet': '<span class="searchmatch">2025</span> FIA <span class="searchmatch">Formula</span> 3 <span class="searchmatch">Championship</span> Drivers&#039; Champion: Rafael Câmara Teams&#039; Champion: Campos Racing Previous 2024 Next 2026 Parent series: <span class="searchmatch">Formula</span> <span class="searchmatch">One</span> World', 'timestamp': '2025-11-29T19:29:27Z'}, {'ns': 0, 'title': '2017 Formula One World Championship', 'pageid': 47234667, 'size': 134834, 'wordcount': 6054, 'snippet': 'Support series: <span class="searchmatch">Formula</span> 2 <span class="searchmatch">Championship</span> GP3 Series Porsche Supercup The 2017 FIA <span class="searchmatch">Formula</span> <span class="searchmatch">One</span> <span class="searchmatch">World</span> <span class="searchmatch">Championship</span> was the 71st season of <span class="searchmatch">Formula</span> <span class="searchmatch">One</span> motor racing', 'timestamp': '2025-12-11T18:35:22Z'}]
